In [ ]:
from pynq import Overlay
from pynq import DefaultIP

In [ ]:
from math import log
class AbstractIP(DefaultIP):
    bindto = None
    
    def __init__(self, description):
        super().__init__(description)
        # Variables User Sets Up
        self.isUL = None
        self.crcLen = None
        self.K = None
        self.msgLen = None
        self.E = None
        self.modScheme = None
        # Internal variables
        self._iIl = None
        self._iBl = None
        self._nMax = None
        self.isConfigured = False
        self._modSchemes = ["QPSK","BPSK","QAM16","QAM64"]
        self._modSchemeBps = {"BPSK":1,"QPSK":2,"QAM16":4,"QAM64":6}
    def _setInternalVar(self,isUL):
        ''' Setup iIl ,iBl and nMax according to uplink or downlink '''
        self.isUL = isUL
        if self.isUL:
            self._iIl = False
            self.iBl = True
            self.nMax = 10
        else:
            self._iIl = True
            self._iBl = False
            self.nMax = 9
            
    def _reset(self):
        # Variables User Sets Up
        self.isUL = None
        self.crcLen = None
        self.K = None
        self.msgLen = None
        self.E = None
        self.modScheme = None
        # Internal variables
        self._bps = None
        self._iIl = None
        self._iBl = None
        self._nMax = None
        self.isConfigured = False

    def _setK(self,K):
        if self.isUL:
            if (K >= 18 and K <= 1023) and not (K >= 26 and K <= 30):
                self.K = K
            else:
                pass # Throw error here and call _reset()
        else:
            if (K >= 36 and K <= 184):
                self.K = K
            else:
                pass # Throw error here and call _reset()
    
    def _setCrcLen(self,crcLen):
        if self.UL:
            if crcLen == 6 or crcLen == 11:
                self.crcLen = crcLen
            else:
                pass # Throw error here and call _reset()
        else:
            if crcLen == 24:
                self.crcLen = crcLen
            else:
                pass # Throw error here and call _reset()
             
    def _setMsgLen(self):
        self.msgLen = self.K - self.crcLen
    
    def _setupMsgSize(self,K,crcLen):
        if self.UL == None:
            pass # Throw error here and call _reset()
        else:
            self._setK(K)
            self._setCrcLen(crcLen)
            self._setMsgLen()
    
    def _setupModScheme(self,modScheme):
        if modScheme.upper() in self._modSchemes:
            self.modScheme = modScheme
            self._bps = self._modSchemeBps[self.modScheme]
        else:
            pass # Throw Error and call _reset   
    
    
    def _setE(self,E):
        temp = log(E,2)
        temp_int = int(temp)
        if E > self.K and temp == temp_int:
           self.E = E
        else: 
            pass # Throw error call _reset
    def _setupE(self,E):
        self._setE(E)


        
        





## Driver for Encoder_IP


In [ ]:
class Encoder_IP(AbstractIP):
    bindto = ['xilinx.com:hls:polarEncoderIP:1.1']
    def __init__(self, description):
        super().__init__(description)
    
    def _writeConfig(self):
        pass
    
    def config(self,isUL, K,crcLen,E,modScheme):    
        #setup internal variables
        self._setInternalVar(isUL)
        # setup K crcLen and msgSize
        self._setupMsgSize(K,crcLen)
        # setup modulation scheme
        self._setupModScheme(modScheme)
        # setup Rate Matched output Length
        self._setupE(E)
    

    

    


## Driver for Rate_Recover_IP


In [ ]:
class RateRecover_IP(AbstractIP):
    bindto = ['xilinx.com:hls:RateRecover:1.1']
    def __init__(self, description):
        super().__init__(description)

    
    def config():
        pass


## Driver for Decoder_IP


In [ ]:
class Decoder_IP(AbstractIP):
    bindto = ['xilinx.com:hls:PolarDecoder:1.1']
    def __init__(self, description):
        super().__init__(description)

    
    def config():
        pass


## Create Overlay 


In [ ]:
ol = Overlay("PolarCodes.bit")

In [ ]:
ol?

## Create INT array from Characters


In [ ]:
def char_to_int_array(string):
    bits = ''
    for charater in string:
        temp = ord(charater)
        bits += (bin(temp)[2:]).zfill(8)
    bits_array = [int(bit,2) for bit in bits]
    return bits_array
